In [ ]:
def vite_sync ():
    !cd ../vite && ls && yarn build
    !cd ../wp && rm -rf dist 
    !cp -r ../vite/dist ../wp/dist
    !cd ../wp && wp-env start
    # print url http://localhost:8888/
    print ( "http://localhost:8888/" )
